Recall, there is a lot of categories. We will use LLMs to assign + group 


Zero Shot Classification:
- Taking pre-trained LLM, without further training, gettin model to assign text to category
- We give model with our text and prompt, and may provide it with categories. 
- This is done by leveraging knowledge about the classes' attributes or relationships learned during pre-training

So how can models do this?
- transformers based models are good at understanding what words are semantically related to one-another due to words seen during training
- somewhere in the training data, they may be exposed to similar prompts 


In [71]:
import pandas as pd
books = pd.read_csv("/Users/naomigong/Coding/Book_Recommender/books_cleaned.csv")

In [72]:
books["categories"].value_counts().reset_index().query('count >= 50')

#another method
book_counts = books["categories"].value_counts() #returns pandas series
book_counts[book_counts > 50] #boolean mask, so returns the one that are greater than 50


categories
Fiction                      2269
Juvenile Fiction              456
Biography & Autobiography     348
History                       226
Literary Criticism            138
Philosophy                    129
Religion                      126
Comics & Graphic Novels       125
Drama                          99
Juvenile Nonfiction            81
Science                        62
Poetry                         58
Business & Economics           56
Literary Collections           53
Social Science                 52
Name: count, dtype: int64

In [73]:
#examining the juvenille fiction category
#selecting the rows in which the category is juvenile fiction
books[books["categories"] == "Juvenile Fiction"]
books[books["categories"].str.contains("fiction", case=False, na=False)] #need na = false just in case there are na values


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...
11,9780006483908,0006483909,Jimmy the Hand,Raymond E. Feist;S. M. Stirling,Fantasy fiction,http://books.google.com/books/content?id=hV4-o...,"Jimmy the Hand, boy thief of Krondor, lived in...",2003.0,3.95,368.0,5579.0,Jimmy the Hand,"9780006483908 Jimmy the Hand, boy thief of Kro..."
15,9780006496878,0006496873,Mystical Paths,Susan Howatch,English fiction,http://books.google.com/books/content?id=by4yt...,1968 finds Nicholas Darrow wrestling with pers...,1996.0,4.23,576.0,1023.0,Mystical Paths,9780006496878 1968 finds Nicholas Darrow wrest...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5677,9783791329284,3791329286,Joseph Cornell,Joseph Cornell;Alison Baverstock,Juvenile Nonfiction,http://books.google.com/books/content?id=KnO8A...,Provides an introduction to the American artis...,2003.0,4.06,27.0,31.0,Joseph Cornell: Secrets in a Box,9783791329284 Provides an introduction to the ...
5683,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...
5684,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...
5686,9788171565641,8171565646,Aspects of the Novel,E. M. Forster,English fiction,http://books.google.com/books/content?id=qWU9P...,"Forster's lively, informed originality and wit...",2004.0,3.83,141.0,10.0,Aspects of the Novel,"9788171565641 Forster's lively, informed origi..."


In [74]:
#create a mapping
category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Children's Fiction",
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Children's Nonfiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction"}

books["simplifed_categories"] = books["categories"].map(category_mapping)
books["simplifed_categories"].value_counts()

simplifed_categories
Fiction                  2551
Nonfiction               1029
Children's Fiction        456
Children's Nonfiction      81
Name: count, dtype: int64

It creates a zero-shot classification pipeline

It internally uses the model and tokenizer you specified (facebook/bart-large-mnli)

This pipeline will:

Take an input sentence (e.g., "This book is about science and magic.")

And compare it against a list of labels (e.g., ["sci-fi", "fantasy", "romance"])

The model predicts how likely the input entails each label → and ranks them accordingly.



In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

model_name = "facebook/bart-large-mnli"
bart_model = AutoModelForSequenceClassification.from_pretrained(model_name)
bart_tokenizer = AutoTokenizer.from_pretrained(model_name)
#for zero shot classification we need to give it the categories
categories = ["Fiction", "Nonfiction"]
pipe = pipeline("zero-shot-classification", model = model_name)


/opt/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Doing the zero shot classification

In [76]:
books.columns
sequence = books.loc[books["simplifed_categories"] == "Fiction", "description"].reset_index(drop=True)[2]
result = pipe(sequence, categories)
print(result)

max_score = max(result['scores'])
ind_max = result['scores'].index(max_score)
print(max_score, ind_max)
print("prediction", result['labels'][ind_max])


def generate_prediction(sequence, categories):
    result = pipe(sequence, categories)

    max_score = max(result['scores'])
    ind_max = result['scores'].index(max_score)
    return result['labels'][ind_max]


{'sequence': "Tricked once more by his wily half-brother, Lysaer, Lord of Light, arrives at the tiny harbor town of Merior to find that Arithon's ship yards have been abandoned and meticulously destroyed, and that the Master of Shadow has disappeared as if into thin air. Meanwhile Arithon and the Mad Prophet Dakar are traveling on foot through the treacherous Kelhorn Mountains towards the Vastmark clans, there to raise further support for his cause. But raising a warhost is a costly business. Is it mere coincidence that Princess Talith—Lysaer's beautiful, headstrong wife—is taken captive and held for a vast ransom by a master brigand? The forces of light and shadow circle and feint, drawing ever closer to a huge conflict. And in the background the Fellowship of Seven Sorcerers and the Koriani Enchantresses watch and plan, and wait.", 'labels': ['Fiction', 'Nonfiction'], 'scores': [0.5352597236633301, 0.46474024653434753]}
0.5352597236633301 0
prediction Fiction


Checking our accuracy

In [77]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simplifed_categories"] == "Fiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_prediction(sequence, categories)]
    actual_cats += ["Fiction"]

for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simplifed_categories"] == "Nonfiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_prediction(sequence, categories)]
    actual_cats += ["Nonfiction"]

100%|██████████| 300/300 [01:56<00:00,  2.58it/s]


In [78]:
prediction_df = pd.DataFrame({"ground_truth": actual_cats, "preds": predicted_cats})
prediction_df

,ground_truth,preds
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction
...,...,...
595,Nonfiction,Nonfiction
596,Nonfiction,Nonfiction
597,Nonfiction,Nonfiction
598,Nonfiction,Nonfiction


In [79]:
import numpy as np
prediction_df["correct_prediction"] = (
    np.where(prediction_df["ground_truth"] == prediction_df["preds"], 1, 0)
    
)
prediction_df["correct_prediction"].value_counts()

correct_prediction
1    467
0    133
Name: count, dtype: int64

In [80]:
prediction_df["correct_prediction"].sum()/len(prediction_df)

0.7783333333333333

We want to use our model to predict the missing categories

In [81]:
isbns = []
missing_predictions = []

missing_books = books.loc[books["simplifed_categories"].isna(), ["isbn13", "description"]].reset_index(drop = True)
missing_books

,isbn13,description
0,9780002261982,A new 'Christie for Christmas' -- a full-lengt...
1,9780006280897,Lewis' work on the nature of love divides love...
2,9780006280934,"""In The Problem of Pain, C.S. Lewis, one of th..."
3,9780006380832,Until Vasco da Gama discovered the sea-route t...
4,9780006470229,A new-cover reissue of the fourth book in the ...
...,...,...
1571,9788125026600,Not only does Nietzsche for Beginners delve in...
1572,9788171565641,"Forster's lively, informed originality and wit..."
1573,9788172235222,On A Train Journey Home To North India After L...
1574,9788173031014,This book tells the tale of a man who goes on ...


In [82]:
for i in tqdm(range(len(missing_books))):
    #now generate the predictions
    sequence = missing_books["description"][i]
    result = generate_prediction(sequence, categories)
    missing_predictions.append(result)
    isbns.append(missing_books["isbn13"][i])



100%|██████████| 1576/1576 [09:28<00:00,  2.77it/s]


In [88]:
#missing prediction dataframe
missing_books_df = pd.DataFrame({"isbn13": isbns, "missing_cat_preds": missing_predictions})
missing_books_df

,isbn13,missing_cat_preds
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1571,9788125026600,Nonfiction
1572,9788171565641,Fiction
1573,9788172235222,Fiction
1574,9788173031014,Nonfiction


In [90]:
#now we want to merge the predicted categories for missing
books = pd.merge(books, missing_books_df, on="isbn13", how = "left")
books["simplifed_categories"] = np.where(books["simplifed_categories"].isna(), books["missing_cat_preds"], books["simplifed_categories"])
books = books.drop(columns = "missing_cat_preds")
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simplifed_categories,missing_cat_preds_x,missing_cat_preds_y
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,NaN,NaN
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,Fiction,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,NaN,NaN
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,Nonfiction,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,Nonfiction,Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5688,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,Nonfiction,Nonfiction
5689,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,Fiction,Fiction
5690,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,NaN,NaN
5691,9789027712059,9027712050,The Berlin Phenomenology,Georg Wilhelm Friedrich Hegel,History,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegel's Philo...,1981.0,0.00,210.0,0.0,The Berlin Phenomenology,9789027712059 Since the three volume edition o...,Nonfiction,NaN,NaN


In [91]:
books.to_csv("final_data_set.csv")